# Tratamiento del dataframe 
Se ha extraido de a base de datos un daframe en formato pickle

## Imports

In [18]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sql_face.tables import *
import os
import plotly.express as px
import plotly.io as pio


## Leer dataframe del disco
se especifica directorio y fichero pkl. No necesitamos la base de datos.

In [19]:
home = os.path.expanduser("~")

input_dir = os.path.join(home, 'video_resources', 'sql_database')
df = pd.read_pickle(os.path.join(input_dir,'df_all_colab_21.pkl'))


In [20]:
df.columns

Index(['identity', 'pitch', 'angle_pitch', 'yaw', 'angle_yaw', 'roll',
       'angle_roll', 'source', 'gender', 'headgear', 'age', 'age_number',
       'glasses', 'type', 'beard', 'emotion', 'other_occlusions', 'race',
       'image_id', 'Detector', 'E_Model', 'Q_model', 'Q_group', 'Quality'],
      dtype='object')

In [21]:
df.source.value_counts()

UTKface       706684
LFW           396830
XQLFW         374130
SCFace        118540
ChokePoint     71559
ForenFace      69118
ENFSI          54560
Name: source, dtype: int64

### Quitamos los Embedding Models
y los registros repetidos

In [22]:
df2 = df.copy()
#df2 = df.loc[df['type'] == 'enfsiImage']
df2.drop('E_Model', axis = 1,inplace = True)
df2 = df2.drop_duplicates(subset = ['image_id','Detector','Q_model'])
df2 = df2.reset_index(drop=True)

In [23]:
len(df2)

415537

In [24]:

df2

,identity,pitch,angle_pitch,yaw,angle_yaw,roll,angle_roll,source,gender,headgear,...,type,beard,emotion,other_occlusions,race,image_id,Detector,Q_model,Q_group,Quality
0,2011-1-unknown,Pitch.FRONTAL,-1.539315,Yaw.FRONTAL,-2.064884,Roll.FRONTAL,1.668600,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.NEUTRAL,False,Race.WHITE,1,mtcnn,ser_fiq,QualityGroup.HIGH,0.632035
1,2011-1,Pitch.FRONTAL,8.485744,Yaw.FRONTAL,7.485447,Roll.FRONTAL,0.792988,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.FEAR,False,Race.LATINO,2,mtcnn,ser_fiq,QualityGroup.VERY_HIGH,0.771966
2,2011-2-unknown,Pitch.FRONTAL,-3.920415,Yaw.FRONTAL,1.548413,Roll.FRONTAL,-0.274244,ENFSI,Gender.MALE,False,...,enfsiImage,True,Emotion.NEUTRAL,False,Race.WHITE,3,mtcnn,ser_fiq,QualityGroup.MEDIUM,0.491990
3,2011-2,Pitch.FRONTAL,-8.626112,Yaw.FRONTAL,7.159441,Roll.HALF_LEANING,-7.312764,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.SAD,False,Race.ASIAN,4,mtcnn,ser_fiq,QualityGroup.HIGH,0.702496
4,2011-3,Pitch.FRONTAL,9.193484,Yaw.FRONTAL,8.528775,Roll.FRONTAL,2.590432,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.SAD,False,Race.WHITE,5,mtcnn,ser_fiq,QualityGroup.LOW,0.053961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415532,0001,None,NaN,None,NaN,None,NaN,ChokePoint,Gender.MALE,None,...,cpVideoFrame,None,Emotion.NEUTRAL,None,Race.ASIAN,133930,mtcnn_serfiq,ser_fiq,QualityGroup.HIGH,0.670448
415533,0001,None,NaN,None,NaN,None,NaN,ChokePoint,Gender.MALE,None,...,cpVideoFrame,None,Emotion.NEUTRAL,None,Race.ASIAN,133931,mtcnn_serfiq,ser_fiq,QualityGroup.HIGH,0.623821
415534,0001,None,NaN,None,NaN,None,NaN,ChokePoint,Gender.MALE,None,...,cpVideoFrame,None,Emotion.SAD,None,Race.ASIAN,133932,mtcnn_serfiq,ser_fiq,QualityGroup.HIGH,0.643205
415535,0001,None,NaN,None,NaN,None,NaN,ChokePoint,Gender.MALE,None,...,cpVideoFrame,None,Emotion.NEUTRAL,None,Race.ASIAN,133933,mtcnn_serfiq,ser_fiq,QualityGroup.MEDIUM,0.555661


In [25]:
df2.Q_model.value_counts()

ser_fiq    243548
tface      171989
Name: Q_model, dtype: int64

### Nos quedamos con Un solo detector y quality Model

In [27]:
df3 = df2.loc[(df2.Detector == 'mtcnn') & (df2.Q_model=='ser_fiq')]
df3.reset_index(drop=True, inplace = True)


In [28]:
df3

,identity,pitch,angle_pitch,yaw,angle_yaw,roll,angle_roll,source,gender,headgear,...,type,beard,emotion,other_occlusions,race,image_id,Detector,Q_model,Q_group,Quality
0,2011-1-unknown,Pitch.FRONTAL,-1.539315,Yaw.FRONTAL,-2.064884,Roll.FRONTAL,1.668600,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.NEUTRAL,False,Race.WHITE,1,mtcnn,ser_fiq,QualityGroup.HIGH,0.632035
1,2011-1,Pitch.FRONTAL,8.485744,Yaw.FRONTAL,7.485447,Roll.FRONTAL,0.792988,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.FEAR,False,Race.LATINO,2,mtcnn,ser_fiq,QualityGroup.VERY_HIGH,0.771966
2,2011-2-unknown,Pitch.FRONTAL,-3.920415,Yaw.FRONTAL,1.548413,Roll.FRONTAL,-0.274244,ENFSI,Gender.MALE,False,...,enfsiImage,True,Emotion.NEUTRAL,False,Race.WHITE,3,mtcnn,ser_fiq,QualityGroup.MEDIUM,0.491990
3,2011-2,Pitch.FRONTAL,-8.626112,Yaw.FRONTAL,7.159441,Roll.HALF_LEANING,-7.312764,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.SAD,False,Race.ASIAN,4,mtcnn,ser_fiq,QualityGroup.HIGH,0.702496
4,2011-3,Pitch.FRONTAL,9.193484,Yaw.FRONTAL,8.528775,Roll.FRONTAL,2.590432,ENFSI,Gender.MALE,False,...,enfsiImage,False,Emotion.SAD,False,Race.WHITE,5,mtcnn,ser_fiq,QualityGroup.LOW,0.053961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58929,064,Pitch.FRONTAL,NaN,Yaw.FRONTAL,NaN,Roll.FRONTAL,NaN,ForenFace,Gender.MALE,True,...,image,None,Emotion.NEUTRAL,None,Race.WHITE,61639,mtcnn,ser_fiq,QualityGroup.LOW,0.014163
58930,073,Pitch.FRONTAL,NaN,Yaw.FRONTAL,NaN,Roll.FRONTAL,NaN,ForenFace,Gender.MALE,False,...,image,None,Emotion.NEUTRAL,None,Race.WHITE,61640,mtcnn,ser_fiq,QualityGroup.MEDIUM,0.426552
58931,001,Pitch.FRONTAL,NaN,Yaw.PROFILE,NaN,None,NaN,ForenFace,Gender.MALE,False,...,image,None,Emotion.SAD,None,Race.WHITE,61641,mtcnn,ser_fiq,QualityGroup.VERY_LOW,0.001988
58932,053,Pitch.FRONTAL,NaN,Yaw.FRONTAL,NaN,Roll.FRONTAL,NaN,ForenFace,Gender.FEMALE,False,...,image,None,Emotion.NEUTRAL,None,Race.WHITE,61642,mtcnn,ser_fiq,QualityGroup.HIGH,0.733252


### Convertimos en string los Enums y nos quedamos con las columnas que queremos

In [31]:
#df3 = df2.copy()
#convertimos a String los Enum
variables = ['gender','yaw','emotion','race']#'Q_group','pitch','roll',
#df_plot = df.Quality
for var in variables:
    df3[var] = df3[var].apply(lambda x: x.value if x else None)


variables += ['source','Quality','Detector', 'Q_model']
df_plot = df3[variables]

In [32]:
df_plot.columns

Index(['gender', 'yaw', 'emotion', 'race', 'source', 'Quality', 'Detector',
       'Q_model'],
      dtype='object')

In [33]:
df_plot[['Q_model','Detector','source']].groupby(['Q_model','Detector']).value_counts()

Q_model  Detector  source   
ser_fiq  mtcnn     UTKface      24059
                   LFW          13233
                   XQLFW        13159
                   SCFace        3882
                   ForenFace     2371
                   ENFSI         2230
dtype: int64

# Separamos por source

In [34]:
dict = {}
for source in np.unique(df_plot.source):
    dict[source]= df_plot.loc[df_plot.source == source]

## definimos la función de ploteo

In [47]:
def plot_sun(source):
    df_x = dict[source]
    df_x.dropna(inplace=True)
    fig = px.sunburst(df_x, path=['gender','race','yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis",
                  range_color=[0, 1],title=source) #values="gender"#"Q_model", "Detector",
    #fig.update_layout(title=source)
    #img_data = pio.to_image(fig,format='png',scale=3)

    pio.write_image(fig,os.path.join(input_dir,source+"_sunburst_21.png"),format='png', scale=6)
    fig.show()

In [ ]:
# df_x = dict[source].drop('race',axis=1)

In [37]:
dict.keys()

dict_keys(['ENFSI', 'ForenFace', 'LFW', 'SCFace', 'UTKface', 'XQLFW'])

## Definimos las sources que queremos y hacemos el plot (y se gueardan en disco)

In [41]:
sources = [ 'ForenFace', 'LFW', 'SCFace',  'XQLFW']

In [48]:
#for source in dict.keys():
for source in sources:
    plot_sun(source)

In [38]:
df = dict['ForenFace']

In [39]:
df

,gender,yaw,emotion,race,source,Quality,Detector,Q_model
56563,Man,straight,neutral,white,ForenFace,0.003972,mtcnn,ser_fiq
56564,Man,straight,neutral,white,ForenFace,0.019352,mtcnn,ser_fiq
56565,Man,straight,sad,white,ForenFace,0.339704,mtcnn,ser_fiq
56566,Man,sideways,happy,white,ForenFace,0.014630,mtcnn,ser_fiq
56567,Woman,straight,happy,white,ForenFace,0.593187,mtcnn,ser_fiq
...,...,...,...,...,...,...,...,...
58929,Man,straight,neutral,white,ForenFace,0.014163,mtcnn,ser_fiq
58930,Man,straight,neutral,white,ForenFace,0.426552,mtcnn,ser_fiq
58931,Man,sideways,sad,white,ForenFace,0.001988,mtcnn,ser_fiq
58932,Woman,straight,neutral,white,ForenFace,0.733252,mtcnn,ser_fiq


In [ ]:
source = 'XQLFW'
df_plot = dict[source]
df_plot.dropna(inplace=True)

In [ ]:
fig = px.sunburst(df_plot, path=['gender','yaw','race','emotion'],   color="Quality",
                  color_continuous_scale="Viridis",title=source) #values="gender"#"Q_model", "Detector",
#fig.update_layout(title=source)

img_data = pio.to_image(fig,format='png',scale=3)

pio.write_image(fig,source+".png",format='png', scale=6)
fig.show()


In [ ]:
type(img_data)

In [ ]:

# Ajustar el tamaño de la figura
plt.figure(figsize=(10, 6))

# Crear el stripplot
sns.swarmplot(x='yaw', hue='emotion', y= 'Quality',  data=df_plot)#,jitter=True,dodge=True,
#sns.scatterplot(x='yaw2', hue='emotion2', y= 'Quality',  data=df2)#,dodge=True,
# Mostrar el gráfico
plt.show()


In [ ]:
sns.swarmplot(x='race', hue='gender', y= 'Quality',  data=df_plot)#,jitter=True,dodge=True,

In [ ]:
import plotly.express as px

fig = px.treemap(df_plot, path=['Q_group', 'Q_model', 'Detector', 'gender'], title="Mosaic Plot")
fig.show()

In [ ]:


# # Tabla de contingencia para Quality_group y Detector
# contingency_table_detector = pd.crosstab(df_plot['Q_group'], df_plot['Detector'])

# # Tabla de contingencia para Quality_group y Quality_model
# contingency_table_quality_model = pd.crosstab(df_plot['Q_group'], df_plot['Q_model'])

# # Tabla de contingencia para Quality_group y yaw
# contingency_table_yaw = pd.crosstab(df_plot['Q_group'], df_plot['yaw'])

# # Tabla de contingencia para Quality_group y gender
# contingency_table_gender = pd.crosstab(df_plot['Q_group'], df_plot['gender'])


# # Tabla de contingencia para Quality_group y emotion
# contingency_table_emotion = pd.crosstab(df_plot['Q_group'], df_plot['emotion'])

# # Tabla de contingencia para Quality_group y race
# contingency_table_race = pd.crosstab(df_plot['Q_group'], df_plot['race'])


In [ ]:

# def norm_table (df):
#     for col in df.columns:
#         df[col] = df[col]/sum(df[col])



In [ ]:
# norm_table(contingency_table_quality_model)


In [ ]:
# norm_table(contingency_table_detector)
# contingency_table_detector

In [ ]:
# norm_table(contingency_table_gender)
# contingency_table_gender

In [ ]:
# norm_table(contingency_table_yaw)
# contingency_table_yaw

In [ ]:
# norm_table(contingency_table_race)
# contingency_table_race

In [ ]:
# norm_table(contingency_table_emotion)
# contingency_table_emotion

In [ ]:
# contingency_table_emotion.plot(kind='bar', subplots=False)
# plt.legend(loc='best')


In [ ]:
# contingency_table_race.plot(kind='bar', subplots=False)
# plt.legend(loc='best')


In [ ]:
# contingency_table_yaw.plot(kind='bar', subplots=False)
# plt.legend(loc='best')


In [ ]:
import matplotlib.pyplot as plt

def plot_stacked_bar(contingency_table, title, xlabel, ylabel):
    ax = contingency_table.plot(kind='bar', stacked=True, figsize=(10, 7))
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc='best')
    plt.show()

# plot_stacked_bar(contingency_table_detector, 'Quality_group vs Detector', 'Quality_group', 'Frequency')
# plot_stacked_bar(contingency_table_quality_model, 'Quality_group vs Quality_model', 'Quality_group', 'Frequency')
# plot_stacked_bar(contingency_table_yaw, 'Quality_group vs yaw', 'Quality_group', 'Frequency')
# plot_stacked_bar(contingency_table_gender, 'Quality_group vs gender', 'Quality_group', 'Frequency')


In [ ]:
df_plot.head()


In [ ]:
import plotly.io as pio

#fig = ...  # tu figura de plotly aquí
#pio.write_image(fig, 'figure.png', format='png', scale=2) 

In [ ]:
# Crear un gráfico sunburst con Plotly
fig1 = px.sunburst(df_plot1, path=['race','gender'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig2 = px.sunburst(df_plot1, path=['yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig3 = px.sunburst(df_plot1, path=['Detector','yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig4 = px.sunburst(df_plot1, path=['gender','race','yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig5 = px.sunburst(df_plot1, path=['Detector','gender','yaw','race','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",

fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()

In [ ]:
# Crear un gráfico sunburst con Plotly
fig1 = px.sunburst(df_plot1, path=['race','gender'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig2 = px.sunburst(df_plot1, path=['yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig3 = px.sunburst(df_plot1, path=['Detector','yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig4 = px.sunburst(df_plot1, path=['gender','race','yaw','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",
fig5 = px.sunburst(df_plot1, path=['Detector','gender','yaw','race','emotion'],   color="Quality",
                  color_continuous_scale="Viridis") #values="gender"#"Q_model", "Detector",

fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()

In [ ]:
tabla1 =df_plot1.groupby(['Q_model','Detector',"gender",'race','yaw','emotion']).agg({"Quality":["count","mean"]})

In [ ]:
type(tabla)

In [ ]:
tabla1.to_excel("tabla_serfiq.xlsx")

In [ ]:
df.angle_yaw

In [ ]:
df_plot.yaw.value_counts()